In [1]:
pip install micromlgen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for micromlgen: filename=micromlgen-1.1.28-py3-none-any.whl size=32173 sha256=9eaf1952a75313bee522667136cf3582586ca14f968958292943ba9504ed19e2
  Stored in directory: /root/.cache/pip/wheels/b5/c1/36/9848f822936171a90ec43d0534cb86a58e1dff20c645e22074
Successfully built micromlgen


In [7]:
from sklearn.model_selection import train_test_split as split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from pandas import read_csv
from micromlgen import port
from sklearn.preprocessing import LabelEncoder

In [8]:
# Load dataset
df = read_csv('https://raw.githubusercontent.com/wooihaw/datasets/main/genders_heights_weights.csv')
df.sample(5)

,Gender,Height,Weight
9781,Female,165.5,64.2
4461,Male,176.8,78.9
6271,Male,173.0,74.3
8830,Female,157.7,56.9
5655,Male,170.0,80.3


In [9]:
from pandas import get_dummies

df2 = get_dummies(df, drop_first=True)
df2.sample(5)

,Height,Weight,Gender_Male
1309,161.1,56.5,0
1886,164.3,59.7,0
8563,163.1,65.4,0
8669,184.9,91.5,1
4848,166.8,68.2,0


In [10]:
# Divide in features and targets
y = df2['Weight'].values  # targets
del df2['Weight']  # remove Weight column

X = df2.values  # features

X_train, X_test, y_train, y_test = split(X, y, random_state=42)

lnr = LinearRegression().fit(X_train, y_train)
print(f"lnr R2 score: {lnr.score(X_test, y_test)}")

lnr R2 score: 0.9057030207072474


In [13]:
# Perform prediction on user input values
h = float(input("Enter height in cm: "))
g = float(input("Enter gender (female->0, male->1): "))
print(f"Predicted as {lnr.predict([[h, g]])} kg")

Enter height in cm: 175
Enter gender (female->0, male->1): 0
Predicted as [75.67340561] kg


In [14]:
# Export to C code
c_code = port(lnr)

with open("linear_regression.h", 'w') as f:
  f.write(c_code)